
<p align="center">
<h1 align="center"> Trabajo Obligatorio - Pensamiento Computacional - 2023 </h1>
</p>
<p align="center">
   <img src="https://github.com/gocolman-uru/UM/blob/obligatorio_pens_computacional/test/static/assets/img/ApUMtes.jpg" width="75" alt="apuntes">
</p>

* Status: accepted 
* Deciders: Gonzalo Colman
* Date: 2022-08-01

Para realizar la ejecución del programa, dejamos el detalle del [Manual de usuario] con sus pasos a seguir. (https://github.com/gocolman-uru/UM/blob/obligatorio_pens_computacional/Manual_de_usuario.md)

Technical Story: Given an rep's comment when the case has been closed, we provide the key tags, risk features with their score, that has been descripted in the comment block. 


## Model Overview
* Problem Family: supervised
* Model Family: POC clustering ---> Multitag
* Model Type: Ngrams & RandomForestClassifier (OneVsRestClassifier)
* Model setup: 
                              - class_weight = 'balanced',
                              - n_estimators = 200, 
                              - max_features = log2,
                              - max_depth = 10,
                              - criterion = 'gini'
                              
                              
* Model Dimension:  

                    def train_model(name_transformer, X, y , estimador):
                        vectores = encoder_transformer(name_transformer,X)
                        clf = estimador.fit(vectores, y)
                        return clf
                        
                        El modelo utiliza SentenceTransformer --> 'multi-qa-mpnet-base-dot-v1'
                    
                    
                    
* Dataset Dimension: **(10346, 23)**
* Test size: **0.15** (1342,23)
* Validation size: **0.15** (1342,23)

* Validation Metric: precision
* Validation Performance: **0.883** --> **general** <font size = 1>(*Notebook 2_model_exploration.ipynb*)</font>

# Dataset
## Classes (subtypes)

In order to achieve the subtype's prediction, first we had to define the correct classes and the definition of what comment is associated to the right casuistry.

We took all the subtypes of fraud prevention, and unified in some ways that in the end we would have only few general classes, grouped by a sort of "theme". For example, we unified bunch of payments cases that corresponse to differents subtypes in one, so we can assert a payment's class in the trainset without adding granularity. 

* Label Version [1.0.0]
* Label date-range: 2022-01-01 to 2022-09-01
* Label cardinality: 9 possible classes. Label encoder field 'gca_subtype_encoder'

> #### Target dataset


                    
                    
> After all, we applied some filters and transformations

                - Exclude null comments
                - Join EV and Restriction fields
                - Only comments with len() > 10
                - Exclude cases with multiple subtypes
                - Exclude RM payment cases
                - Exclude EV = (DOK|DM) with null Restriction
                - Exclude Restriction = (CBK) with another EV rather than Prev_inhab_perm 
                - Classes unified (details in settings.py)
                - Exclude EV = POI & Otros
                - Exclude EV = (DM) and Restriction = Fraude_ml
                - Exclude EV = None and Restriction = Ret_mp
                - Only consider cases that had been closed with Deactive resolution
                - Exclude generic comments (details in settings.py)
                - Exclude two script comments (details in settings.py)
                - Exclude Bacen and CC cases
    
        

## Considerations
We sample the trainset and balanced our Classes with 100k records per class. 

## Feature Summary
* Number of features: 128 word embeddings
* Name of target column: **gca_subtype_encoder**


## Text pre-processing

* ETL stage: [None]
* Model building stage: [Text to lower, remove accents, remove punctuation, Join comments EV+Restriction]



## Context and Problem Statement

This kind of situation happens in the Manual Review universe. It's a matter of time that automatic decisions will scale and PF's operation teams lean on strong rules and model outputs more than yesterday. 
When an user's contact incomes in the manual review queue, a set of strong rules (based on multiple features) classified and redirects the case into the right team. At the bottom of this "if sentence", there is always an "else". There is where the problem lives. Fraud prevention's Representants are assigned to this task to analyze and manually classified that cases.

## Decision Drivers 

We know since the beginning of the project that it would be an NLP problem. Our input are the EV and Restriction's comment, so we ended to consider the AutoKeras text classifier. We walked this journey as a one team, from the analytics squad to the data scientists involved in this project. Weekly huddles and slack channels were crucial. 